In [1]:
import os
os.environ["JAVA_HOME"] = "jdk-11"

In [2]:
from pyspark.sql import functions as f
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.types import *

spark = SparkSession \
      .builder \
      .appName('various') \
      .master("local[*]") \
      .getOrCreate()\

sc = spark._sc

spark.version

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/30 16:29:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


'3.2.0'

In [3]:
stream_dir = 'individual_heroes'

schema = StructType([StructField('Name',StringType(),True),
                     StructField('Alignment',StringType(),True),
                     StructField('Intelligence',DecimalType(10, 2),True),
                     StructField('Strength',DecimalType(10, 2),True),
                     StructField('Speed',IntegerType(),True),
                     StructField('Durability',DecimalType(10, 2),True),
                     StructField('Power',DecimalType(10, 2),True),
                     StructField('Combat',DecimalType(10, 2),True),
                     StructField('Total',DecimalType(10, 2),True),
                     StructField('Id',IntegerType(),True)])

df = spark.readStream\
  .schema(schema)\
  .csv(stream_dir)

In [4]:
query = df\
  .writeStream\
  .outputMode("console") \

  .outputMode("append")\
  .queryName("heroes") \
  .start() 

21/12/30 16:33:54 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-e2e5cabc-db15-4f89-99d3-106566ff165a. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
21/12/30 16:33:55 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [6]:
from IPython.display import display, clear_output
import time

while True:
    clear_output(wait=True)
    display(query.status)
    display('Total number of heroes: ' + str(spark.sql('SELECT count(*) as count FROM heroes').first()['count']))
    
    heroes_stats = spark.sql('''
                                select
                                    Alignment,
                                    count(*) as Frequency
                                from heroes
                                group by Alignment
                                order by 1 desc
                             ''')
    
    display('Showing stats for heroes:')
    display(heroes_stats.show())

    time.sleep(1)

{'message': 'Getting offsets from FileStreamSource[file:/home/jovyan/individual_heroes]',
 'isDataAvailable': False,
 'isTriggerActive': True}

KeyboardInterrupt: 